# Examples of Complex Blocking Programs for Delex

Here we give several examples of complex blocking programs for Delex. 

To begin, we need to download three datasets from GitHub. Navigate to the dblp_acm folder [here](https://github.com/anhaidgroup/delex/tree/main/examples/data/dblp_acm). Click on 'gold.parquet' and click the download icon at the top. Repeat this for 'table_a.parquet' and 'table_b.parquet'. Now move all these into a directory on your local machine called 'dblp_acm'. Then download this Python notebook [here](https://github.com/anhaidgroup/delex/tree/main/examples/data/program_examples.ipynb), and move it into the 'dblp_acm' folder you just created.

We will gloss over the setup in the initial cells since they are explained in the basic example notebook [here](./basic_example.ipynb).

## Setup

In [ ]:
from pathlib import Path

import sys
sys.path.append(str(Path().resolve().parent))
import os
os.environ['PYTHONPATH'] = str(Path().resolve().parent)

from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


from delex.lang.predicate import (
        BM25TopkPredicate,
        JaccardPredicate,
        EditDistancePredicate,
        SmithWatermanPredicate,
        JaroPredicate, 
        JaroWinklerPredicate, 
        CosinePredicate, 
        OverlapCoeffPredicate,
        ExactMatchPredicate
)

from delex.lang import BlockingProgram, DropRule, KeepRule
from delex.tokenizer import StrippedWhiteSpaceTokenizer, QGramTokenizer, AlphaNumericTokenizer
from delex.execution.plan_executor import PlanExecutor
import operator
import psutil

# enable pyarrow execution, recommended for better performance
conf = SparkConf()\
        .set('spark.sql.execution.arrow.pyspark.enabled',  'true')

# initialize a local spark context
spark = SparkSession.builder\
                    .master('local[*]')\
                    .config(conf=conf)\
                    .appName('Basic Example')\
                    .getOrCreate()

# path to the test data directory
data_path = (Path().resolve() / 'data' / 'dblp_acm').absolute()

# table to be indexed, generally this should be the table with fewer rows
index_table_path = data_path / 'table_a.parquet'
# table for searching
search_table_path = data_path / 'table_b.parquet'
# the ground truth, i.e. the correct matching pairs
gold_path = data_path / 'gold.parquet'

# read all the data as spark dataframes
index_table = spark.read.parquet(f'file://{str(index_table_path)}')
search_table = spark.read.parquet(f'file://{str(search_table_path)}')
gold = spark.read.parquet(f'file://{str(gold_path)}')

index_table.printSchema()

In [ ]:
executor = PlanExecutor(
        index_table=index_table, 
        search_table=search_table,
        optimize=False,
        estimate_cost=False,
)

# Function to compute and print basic stats
def execute_and_compute_stats(prog):
    candidates, _ = executor.execute(prog, search_table_id_col='_id', projection=['_id'])
    # unroll the output
    pairs = candidates.select(
                    F.explode('id1_list').alias('a_id'),
                    F.col('id2').alias('b_id')
                )
    # total number 
    n_pairs = pairs.count()
    true_positives = gold.intersect(pairs).count()
    recall = true_positives / gold.count()
    precision = true_positives / n_pairs if n_pairs else 0.0
    f1_score = (2 * recall * precision) / (recall + precision) if  (recall + precision) > 0 else 0.0
    print(f'n_pairs : {n_pairs}')
    print(f'true_positives : {true_positives}')
    print(f'recall : {recall}')
    print(f'precision : {precision}')
    print(f'F1 : {f1_score}')

## Blocking Program Examples

Below we give examples of blocking programs of increasing complexity. We also provide some suggestions for tuning. We begin with the following simple blocking program:



In [ ]:
prog = BlockingProgram(
        keep_rules = [
                KeepRule([
                    JaccardPredicate('title', 'title', QGramTokenizer(3), operator.ge, .4)
                ])
            ],
        drop_rules = [],
    )

execute_and_compute_stats(prog)

In terms of SQL this program is equivalent to

```SQL
SELECT A.id, B.id
FROM index_table as A, search_table as B
WHERE jaccard_3gram(A.title, B.title) >= .4
```

We can see that this program has quite high recall but almost 50% of the output is non-matching pairs. While this is fine in most cases since we typically run a matching algorithm afterwards which would filter the non-matching pairs out, we can certainly do better with just the blocking rules. What happens if we increase the threshold? 

In [ ]:
prog = BlockingProgram(             
            keep_rules = [
                KeepRule([
                    JaccardPredicate('title', 'title', QGramTokenizer(3), operator.ge, .8)
                ])
            ],
        drop_rules = [],
    )
execute_and_compute_stats(prog)

We can see that increasing the threshold dropped the recall by a few percent but improve the precision significantly. (Here recall is the fraction of true matches that occur in the blocking output, and precision is the fraction of pairs in the blocking output that are true matches. Ideally we want both high recall and high precision. But this is difficult, so we typically aim for high recall and as high precision as we can.)  Of course, we can also increase precision by adding more predicates.

In [ ]:
prog = BlockingProgram(
        keep_rules = [
                KeepRule([ 
                    JaccardPredicate('title', 'title', QGramTokenizer(3), operator.ge, .4),
                    CosinePredicate('authors', 'authors', AlphaNumericTokenizer(), operator.ge, .3)
                ])
            ],
        drop_rules = [],
    )

execute_and_compute_stats(prog)

By adding a simple predicate on the `authors` field, we can increase the precision with a much smaller drop in recall. We could also add a drop rule with a single predicate which is **almost** equivalent to the program above.

In [ ]:
prog = BlockingProgram(
        keep_rules = [
                KeepRule([ 
                    JaccardPredicate('title', 'title', QGramTokenizer(3), operator.ge, .4)
                ])
            ],
        drop_rules = [
                DropRule([
                    CosinePredicate('authors', 'authors', AlphaNumericTokenizer(), operator.lt, .3)
                ])
            ],
    )

execute_and_compute_stats(prog)

Notice that the outputs are slightly different? This is due to how nulls are handled. In particular, if either record field is `None` the comparison function always returns `np.nan` which in turn means the predicate always evaluates to `False`. In terms of sql the first program is equivalent to 

```SQL
SELECT A.id, B.id
FROM index_table as A, search_table as B
WHERE jaccard_3gram(A.title, B.title) >= .4 AND cosine_alnum(A.authors, B.authors) >= .3
```


In contrast the second function is equivalent to
```SQL
SELECT A.id, B.id
FROM index_table as A, search_table as B
WHERE (jaccard_3gram(A.title, B.title) >= .4
EXCEPT 
SELECT A.id, B.id
FROM index_table as A, search_table as B
WHERE cosine_alnum(A.authors, B.authors) < .3
```

Switching back to improving precision, we can also improve precision by adding more predicates.

In [ ]:
prog = BlockingProgram(
        keep_rules = [
                KeepRule([ 
                    JaccardPredicate('title', 'title', QGramTokenizer(3), operator.ge, .4),
                    CosinePredicate('authors', 'authors', AlphaNumericTokenizer(), operator.ge, .3),
                    ExactMatchPredicate('year', 'year', lowercase=True, invert=False),
                ])
            ],
        drop_rules = [],
    )

execute_and_compute_stats(prog)

Just be careful when adding predicates as a bad single predicate can be the difference between high precision + high recall and no output.

In [ ]:
prog = BlockingProgram(
        keep_rules = [
                KeepRule([ 
                    JaccardPredicate('title', 'title', QGramTokenizer(3), operator.ge, .4),
                    CosinePredicate('authors', 'authors', AlphaNumericTokenizer(), operator.ge, .3),
                    ExactMatchPredicate('year', 'year', lowercase=True, invert=False),
                    ExactMatchPredicate('venue', 'venue', lowercase=True, invert=False),

                ])
            ],
        drop_rules = [],
    )

execute_and_compute_stats(prog)

So far we have only looked at threshold based predicates and equality predicates. One very powerful type of predicate is top-k based predicates. BM25 based top-k is provided as a built in (BM25 is a common TFIDF-based ranking function used for full-text search). 

In [ ]:
prog = BlockingProgram(
        keep_rules = [
                KeepRule([
                    BM25TopkPredicate('title', 'title', 'standard', 10)
                ]),
            ],
        drop_rules = [
        ],
    )

execute_and_compute_stats(prog)

As you can see we can get very high recall with just a single predicate. We can get the last few matching pairs by increasing k: 

In [ ]:
prog = BlockingProgram(
        keep_rules = [
                KeepRule([
                    BM25TopkPredicate('title', 'title', 'standard', 20)
                ]),
            ],
        drop_rules = [
        ],
    )

execute_and_compute_stats(prog)

Top-k predicates work the same as threshold or equality based. For example we can use two in a single keep rule.

In [ ]:
prog = BlockingProgram(
        keep_rules = [
                KeepRule([
                    BM25TopkPredicate('title', 'title', 'standard', 20), 
                    BM25TopkPredicate('authors', 'authors', 'standard', 20), 
                ]),
            ],
        drop_rules = [
        ],
    )

execute_and_compute_stats(prog)

Of course we can also add drop rules to refine the output.

In [ ]:
prog = BlockingProgram(
        keep_rules = [
                KeepRule([
                    BM25TopkPredicate('title', 'title', 'standard', 20), 
                ]),
                KeepRule([
                    BM25TopkPredicate('authors', 'authors', 'standard', 20), 
                ]),
            ],
        drop_rules = [
            DropRule([
                JaccardPredicate('title', 'title', QGramTokenizer(3), operator.lt, .3)
            ]),
              DropRule([
                OverlapCoeffPredicate('authors', 'authors', AlphaNumericTokenizer(), operator.lt, .3)
            ]),
            DropRule([
                ExactMatchPredicate('venue', 'venue', invert=True, lowercase=True),
                ExactMatchPredicate('year', 'year', invert=True)
            ])
        ],
    )

execute_and_compute_stats(prog)

## Blocking Program Validation

We now discuss blocking program validation, in particular the set of conditions that must be met for a blocking program to be valid and able to be executed. We break this down into the following sections:

1. Indexable and Streamable `Predicates`
2. `KeepRule` Validation
3. `DropRule` Validation
4. `BlockingProgram` Validation

### 1. Indexable and Streamable Predicates

Predicates have two possible ways that they can be executed, either with an index or in a streaming fasion. To determine the way a predicate can be executed the `Predicate` base class provides two attributes, `indexable` and `streamable`. A predicate can be indexed if `pred.indexable == True` and a predicate can be streamed if `pred.streamable == True`. These attributes are essential for generating an effecient execution plan and are used to validate keep rules, drop rules, and blocking programs.

Some predicates are both `streamable` and `indexable`.

In [ ]:
pred = JaccardPredicate('title', 'title', QGramTokenizer(3), operator.ge, .4)
print(f'{pred.streamable=}')
print(f'{pred.indexable=}')

Others are only `indexable` (typically top-k based predicates).

In [ ]:
pred = BM25TopkPredicate('title', 'title', 'standard', 20) 
print(f'{pred.streamable=}')
print(f'{pred.indexable=}')

Some are only `streamable`. 

In [ ]:
pred = EditDistancePredicate('venue', 'venue', operator.gt, .8)
print(f'{pred.streamable=}')
print(f'{pred.indexable=}')

Note that parameters to the predicate may determine if it is `indexable` or not. For example, `JaccardPredicate` is `indexable` only if the operator is `>=` or `>`.

In [ ]:
pred = JaccardPredicate('title', 'title', QGramTokenizer(3), operator.le, .4)
print(f'{pred.streamable=}')
print(f'{pred.indexable=}')

### 2. KeepRule Validation

A `KeepRule` must satisfy the following criteria:
1. Must contain at least one `indexable` `Predicate`

In [ ]:
rule = KeepRule([
            BM25TopkPredicate('title', 'title', 'standard', 20),
            EditDistancePredicate('venue', 'venue', operator.gt, .8)
        ])
# OK

In [ ]:
rule = KeepRule([
            #BM25TopkPredicate('title', 'title', 'standard', 20),
            EditDistancePredicate('venue', 'venue', operator.gt, .8)
        ])
# ERROR EditDistancePredicate is not indexable

### 3. DropRule Validation

A `DropRule` must satisfy the following criteria:
1. Must contain at least one `Predicate`
2. All `Predicates` in the `DropRule` must be `streamable`

In [ ]:
rule = DropRule([
        JaccardPredicate('title', 'title', QGramTokenizer(3), operator.le, .4)        
    ])
# OK

In [ ]:
rule = DropRule([
        BM25TopkPredicate('title', 'title', 'standard', 20),
        JaccardPredicate('title', 'title', QGramTokenizer(3), operator.le, .4)        
    ])
# ERROR BM25TopkPredicate is not streamable

In [ ]:
rule = DropRule([ ])
# ERROR empty drop rule

### 4. Blocking Program Validation

A `BlockingProgram` must satisfy the following criteria:
1. All `KeepRules` must be valid
2. All `DropRules` must be valid
3. Must contain at least one `KeepRule`

In [ ]:
prog = BlockingProgram(
        keep_rules = [
                KeepRule([
                    BM25TopkPredicate('title', 'title', 'standard', 20), 
                ]),
            ],
        drop_rules = [
            DropRule([
                JaccardPredicate('title', 'title', QGramTokenizer(3), operator.le, .4)
            ]),
        ],
    )
# OK

In [ ]:
prog = BlockingProgram(
        keep_rules = [
                KeepRule([
                    BM25TopkPredicate('title', 'title', 'standard', 20), 
                ]),
            ],
        drop_rules = [
        ],
    )
# OK don't need to provide drop_rules

In [ ]:
prog = BlockingProgram(
        keep_rules = [],
        drop_rules = [
            DropRule([
                JaccardPredicate('title', 'title', QGramTokenizer(3), operator.le, .4)
            ]),
        ],
    )
# ERROR must provide at least one KeepRule